In [1]:
# Carga de la red LSTM pequeña para generar texto
import sys
import numpy
import tensorflow as tf

In [2]:
# load file
filename = "discurso.txt"
text = open(filename, encoding="utf-8").read()


In [16]:
# clean text
text.replace('\n', '')
text = text.lower()

In [17]:
# output values
vocab = sorted(set(text))
# Dict (ids) chars .get_vocabulary(). Get char by index
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [18]:
# convert all text to id, is necessary used numbers
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(25328,), dtype=int64, numpy=array([23, 37, 38, ..., 27, 19, 37], dtype=int64)>

In [19]:
# Dict (chars) ids
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars_from_ids

In [20]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

e
s
t
a
m
o
s
 
c
e


In [24]:
ids_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [25]:
# print sequence
def printSequence(sequences):
    for seq in sequences:
        print(chars_from_ids(seq))
    for seq in sequences:
        print(text_from_ids(seq).numpy())

# get char from ids
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [26]:
# Sequences has tensors with 100 caracteres
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

printSequence(sequences.take(2))

tf.Tensor(
[b'e' b's' b't' b'a' b'm' b'o' b's' b' ' b'c' b'e' b'l' b'e' b'b' b'r'
 b'a' b'n' b'd' b'o' b' ' b'e' b's' b't' b'e' b' ' b'e' b'v' b'e' b'n'
 b't' b'o' b' ' b'd' b'e' b' ' b'n' b'u' b'e' b'v' b'o' b' ' b'e' b'n'
 b' ' b'n' b'u' b'e' b's' b't' b'r' b'a' b' ' b's' b'e' b'd' b'e' b' '
 b'c' b'o' b'r' b'p' b'o' b'r' b'a' b't' b'i' b'v' b'a' b' ' b'd' b'e'
 b' ' b'b' b'o' b'a' b'd' b'i' b'l' b'l' b'a' b' ' b'p' b'a' b'r' b'a'
 b' ' b'm' b'i' b'n' b'i' b'm' b'i' b'z' b'a' b'r' b' ' b'l' b'o' b's'
 b' ' b'r' b'i'], shape=(101,), dtype=string)
tf.Tensor(
[b'e' b's' b'g' b'o' b's' b' ' b's' b'a' b'n' b'i' b't' b'a' b'r' b'i'
 b'o' b's' b'\n' b'q' b'u' b'e' b' ' b'p' b'o' b'd' b'r' b'\xc3\xad' b'a'
 b' ' b'c' b'o' b'n' b'l' b'l' b'e' b'v' b'a' b'r' b' ' b'e' b'l' b' '
 b'd' b'e' b's' b'p' b'l' b'a' b'z' b'a' b'm' b'i' b'e' b'n' b't' b'o'
 b' ' b'd' b'e' b' ' b'n' b'u' b'e' b's' b't' b'r' b'o' b's' b' ' b'e'
 b'q' b'u' b'i' b'p' b'o' b's' b' ' b'y' b' ' b'a' b'c' b'c' b'i' b'o'
 b'n' 

In [27]:
# Get input (x) and target (y)
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [28]:
# dataset ready for model
dataset = sequences.map(split_input_target)
dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [29]:
for input_example, target_example in dataset.take(1):
    
    print("Input :", text_from_ids(input_example).numpy())    
    print("Input :", input_example.numpy().shape)
    print("Target:", text_from_ids(target_example).numpy())

Input : b'estamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los r'
Input : (100,)
Target: b'stamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los ri'


In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [14]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch.shape, "# (batch_size, sequence_length, vocab_size)")
    print(target_example_batch.shape)

(64, 100) # (batch_size, sequence_length, vocab_size)
(64, 100)


In [15]:
# define la LSTM model
model = tf.keras.Sequential()
#model.add(tf.keras.layers.Embedding(len(vocab), 100, input_length=100))
model.add(tf.keras.layers.LSTM(254,input_shape=[64,100, 1]))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(len(vocab), activation='softmax'))

ValueError: Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 64, 100, 1)

In [84]:
# Carga de los pesos de la red
opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt)
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 254)               263144    
                                                                 
 dropout_11 (Dropout)        (None, 254)               0         
                                                                 
 dense_11 (Dense)            (None, 50)                12750     
                                                                 
Total params: 275,894
Trainable params: 275,894
Non-trainable params: 0
_________________________________________________________________


In [85]:
model.fit(dataset, epochs=20, batch_size=BATCH_SIZE)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\Matu\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Matu\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Matu\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Matu\anaconda3\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Matu\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Matu\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 214, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{

In [1]:
# Carga de la red LSTM pequeña para generar texto
import sys
import numpy
import tensorflow as tf
import numpy as np

In [2]:
# load file
filename = "discurso.txt"
text = open(filename, encoding="utf-8").read()
text

"estamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los riesgos sanitarios\nque podría conllevar el desplazamiento de nuestros equipos y accionistas a\ncantabria deseamos volver pronto a santander\ny me gustaría comenzar enviando un mensaje de apoyo al pueblo ucraniano que está sufriendo esta terrible catástrofe\nhumanitaria a todos los que han perdido familiares y amigos y a los que han\ntenido que abandonar sus hogares ahora es el momento de apoyar como en los dos\núltimos años de crisis sanitaria santander y nuestros equipos a los que\nquiero agradecer su continuo compromiso y apoyo están apoyando a las personas ya\nlas empresas en los momentos más difíciles y estoy especialmente orgullosa del\ncorredor humanitario que hemos organizado para trasladar a más de 180 refugiados desde polonia hasta españa\njunto con el gobierno español y muchas ongs así como la utilización del\nedificio el solaruco aquí mismo aquí en nuestra sede en nuestra cas

In [3]:
# clean text
text = text.replace('\n', ' ')
text = text.lower()
text

"estamos celebrando este evento de nuevo en nuestra sede corporativa de boadilla para minimizar los riesgos sanitarios que podría conllevar el desplazamiento de nuestros equipos y accionistas a cantabria deseamos volver pronto a santander y me gustaría comenzar enviando un mensaje de apoyo al pueblo ucraniano que está sufriendo esta terrible catástrofe humanitaria a todos los que han perdido familiares y amigos y a los que han tenido que abandonar sus hogares ahora es el momento de apoyar como en los dos últimos años de crisis sanitaria santander y nuestros equipos a los que quiero agradecer su continuo compromiso y apoyo están apoyando a las personas ya las empresas en los momentos más difíciles y estoy especialmente orgullosa del corredor humanitario que hemos organizado para trasladar a más de 180 refugiados desde polonia hasta españa junto con el gobierno español y muchas ongs así como la utilización del edificio el solaruco aquí mismo aquí en nuestra sede en nuestra casa para aloj

In [4]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  25328
Total Vocab:  49


In [6]:
# prepare the dataset of input to output pairs encoded as integers
maxlen = 60
step = 3
sentences = []
next_chars = []
seq_length = 100
for i in range(0, n_chars - maxlen - maxlen, step):
    seq_in = text[i:i + seq_length-1]
    seq_out = text[i + maxlen:i + maxlen + maxlen]
    sentences.append([char_to_int[char] for char in seq_in])
    next_chars.append(char_to_int[seq_out] for char in seq_out)
    n_patterns = len(sentences)
print("Total Patterns: ", n_patterns)
print("{} x {}".format(len(sentences),len(sentences[1])))

Total Patterns:  8403
8403 x 99


In [7]:
dataX = np.array([[[0 for k in range(0,len(chars),1)] for j in range(0,seq_length,1)] for i in range(0,len(dataXSequences),1)])


NameError: name 'dataXSequences' is not defined

In [95]:
text[60:60]

''

In [44]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalizacion
X = X / float(n_vocab)
# codificacion en caliente de la variable de salida
y = tf.keras.utils.to_categorical(dataY)
print(X.shape)

Total Characters:  25328
Total Vocab:  50
Total Patterns:  25268
(25268, 60, 1)


In [ ]:
# define la LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(128, input_shape=(X.shape[1], X.shape[2])))
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))

# Carga de los pesos de la red
opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)

In [39]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

In [35]:
model.fit(X,y, epochs=1)

789/789 [==============================] - 191s 239ms/step - loss: 19.4773


In [36]:
# toma una semilla aleatoria
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Semilla:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
  #print(prediction)
  #print(index)
print("\nHecho.")

Semilla:
" ividendo en efectivo bruto de 515 céntimos de euro por cada acción pagadero a partir del 2 de mayo
d "
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii